In [ ]:
import rasterio
import os
import subprocess # To run gdal commands

#==========================CONFIGURE OF ALIGNING RASTERS=======================
data_dir = "/storage/home/hcoda1/4/hyu483/scratch/no_heat/SOLWEIG/Input/Null_Rasters/4"

bDSM_path = data_dir + '/DSM_aligned.tif'
output_dir = data_dir + '/aligned_Rasters' # Where cleaned/aligned rasters will be saved
os.makedirs(output_dir, exist_ok=True)

# List of other input rasters to process
rasters_to_align = {
    'bDSM': data_dir + '/DSM_aligned.tif',
    'cDSM': data_dir + '/cDSM_aligned.tif',
    'DEM': data_dir + '/DEM_aligned.tif',
    'wall_aspect': data_dir + '/wall_aspect_merged_aligned.tif',
    'wall_height': data_dir + '/wall_height_merged_aligned.tif',
    'lc': data_dir + '/LULC_6491_5class_aligned.tif'
}

# parameters for no data
NO_DATA = -9999

svf_extract_dir = data_dir + '/SVF' # Directory to extract SVF files
svf_aligned_output_base_dir  = output_dir + '/SVF'
os.makedirs(svf_aligned_output_base_dir, exist_ok=True)

#==========================CONFIGURE OF ALIGNING RASTERS=======================

# Get target parameters from bDSM
target_crs = None
target_res_x = None
target_res_y = None
target_bounds = None
target_width = None
target_height = None
target_nodata = None # Important for consistency

try:
    with rasterio.open(bDSM_path) as src:
        target_crs = src.crs
        target_res_x, target_res_y = src.res
        target_bounds = src.bounds
        target_width = src.width
        target_height = src.height
        target_nodata = NO_DATA # 0 is the desired nodata for all aligned rasters # If bDSM has no nodata and you want no nodata in outputs, set to src.nodata
        print(f"Target Parameters (from {os.path.basename(bDSM_path)}):")
        print(f"  CRS: {target_crs}")
        print(f"  Resolution: {target_res_x}, {target_res_y}")
        print(f"  Bounds: {target_bounds}")
        print(f"  Dimensions (W x H): {target_width} x {target_height}")
        print(f"  NoData Value: {target_nodata}")

except rasterio.errors.RasterioIOError as e:
    print(f"ERROR: Could not open bDSM file at {bDSM_path}. Exiting. {e}")
    exit()

print("\n--- Aligning Rasters ---")

for name, input_path in rasters_to_align.items():
    if not os.path.exists(input_path):
        print(f"WARNING: Input file not found at {input_path}. Skipping alignment for {name}.")
        continue
        
    output_path = os.path.join(output_dir, f'{os.path.basename(input_path).replace(".tif", ".tif")}')

    if os.path.exists(output_path):
        print(f"Skipping alignment for {os.path.basename(input_path)} as aligned output already exists.")
        continue 

    print(f"\nProcessing {name} ({os.path.basename(input_path)}) -> {os.path.basename(output_path)}")

    # Construct the gdalwarp command
    gdal_command = [
        'gdalwarp',
        '-overwrite',                  # Overwrite output file if it exists
        '-t_srs', str(target_crs),     # Target CRS
        '-tr', str(target_res_x), str(target_res_y), # Target resolution (pixel size)
        '-te', # Target extent (xmin, ymin, xmax, ymax)
        str(target_bounds.left), str(target_bounds.bottom), str(target_bounds.right), str(target_bounds.top),
        '-ts', str(target_width), str(target_height), # Target dimensions (pixels)
        '-r', 'bilinear',              # Resampling method (e.g., bilinear, cubic, near)
                                       # 'near' for categorical data like LC, 'bilinear' for continuous data
        # Handle NoData: Set output NoData to match bDSM, and tell gdalwarp what input nodata is
        # If input has no nodata, -srcnodata will have no effect.
        # If bDSM has no nodata, then -dstnodata is omitted.
    ]

    # Dynamically add -srcnodata and -dstnodata
    with rasterio.open(input_path) as src_input:
        current_nodata = src_input.nodata
        if current_nodata is not None:
            gdal_command.extend(['-srcnodata', str(current_nodata)])
        if target_nodata is not None:
            if name == 'lc':
                gdal_command.extend(['-dstnodata', '0'])
            else:
                gdal_command.extend(['-dstnodata', str(target_nodata)])
        else:
            # If target bDSM has no nodata, ensure output also has no nodata,
            # or choose a suitable nodata value for the output if needed.
            # For simplicity, if bDSM has no nodata, we won't set -dstnodata,
            # and existing nodata will be reprojected to a value.
            pass

    # Special handling for Land Cover (lc) - use nearest neighbor resampling
    if name == 'lc':
        gdal_command[gdal_command.index('-r') + 1] = 'near' # Change resampling method for LC
        

    gdal_command.extend([input_path, output_path])

    try:
        # Run the gdalwarp command
        print(f"  Running: {' '.join(gdal_command)}")
        result = subprocess.run(gdal_command, check=True, capture_output=True, text=True)
        print("  GDAL Warp Output:\n", result.stdout)
        if result.stderr:
            print("  GDAL Warp Errors (if any):\n", result.stderr)

        # Verify output
        with rasterio.open(output_path) as aligned_src:
            print(f"  Aligned {name} CRS: {aligned_src.crs == target_crs}")
            print(f"  Aligned {name} Shape: {aligned_src.shape == (target_height, target_width)}")
            print(f"  Aligned {name} Res: {aligned_src.res == (target_res_x, target_res_y)}")
            # Check bounds within a tolerance due to floating point precision
            bounds_match = all(abs(getattr(aligned_src.bounds, attr) - getattr(target_bounds, attr)) < 1e-6 for attr in ['left', 'bottom', 'right', 'top'])
            print(f"  Aligned {name} Bounds Match: {bounds_match}")

    except subprocess.CalledProcessError as e:
        print(f"ERROR: GDAL Warp failed for {name}: {e}")
        print("  STDOUT:", e.stdout)
        print("  STDERR:", e.stderr)
    except Exception as e:
        print(f"An unexpected error occurred during processing {name}: {e}")

print("\n--- Aligning SVF Rasters ---")
for root, _, files in os.walk(svf_extract_dir):
            for file in files:
                if file.lower().endswith(('.tif', '.tiff', '.img')): # Add other raster extensions if needed
                    svf_raster_path = os.path.join(root, file)
                    svf_output_file_path = os.path.join(svf_aligned_output_base_dir, f'{os.path.basename(svf_raster_path).replace(".tif", ".tif")}')
                    
                    if os.path.exists(svf_output_file_path):
                        print(f"Skipping alignment for {os.path.basename(svf_raster_path)} as aligned output already exists.")
                        continue 
                    
                    print(f"\nProcessing {name} ({os.path.basename(svf_raster_path)}) -> {os.path.basename(svf_output_file_path)}")
                    svf_gdal_command = [
                        'gdalwarp',
                        '-overwrite',
                        '-t_srs', str(target_crs),
                        '-tr', str(target_res_x), str(target_res_y),
                        '-te',
                        str(target_bounds.left), str(target_bounds.bottom), str(target_bounds.right), str(target_bounds.top),
                        '-ts', str(target_width), str(target_height),
                        '-r', 'bilinear', # SVF values are continuous, so bilinear is suitable
                    ]
                    try:
                        with rasterio.open(svf_raster_path) as src_svf: # Use a new variable for SVF source
                            current_nodata_svf = src_svf.nodata
                            if current_nodata_svf is not None:
                                svf_gdal_command.extend(['-srcnodata', str(current_nodata_svf)])
                            if target_nodata is not None:
                                svf_gdal_command.extend(['-dstnodata', str(target_nodata)])
        
                        svf_gdal_command.extend([svf_raster_path, svf_output_file_path])
        
                        # Run the gdalwarp command for SVF
                        print(f"  Running: {' '.join(svf_gdal_command)}")
                        result = subprocess.run(svf_gdal_command, check=True, capture_output=True, text=True)
                        print("  GDAL Warp Output:\n", result.stdout)
                        if result.stderr:
                            print("  GDAL Warp Errors (if any):\n", result.stderr)
        
                        # Verify output for SVF
                        with rasterio.open(svf_output_file_path) as aligned_src_svf: # Use a new variable for aligned SVF source
                            print(f"  Aligned SVF CRS: {aligned_src_svf.crs == target_crs}")
                            print(f"  Aligned SVF Shape: {aligned_src_svf.shape == (target_height, target_width)}")
                            print(f"  Aligned SVF Res: {aligned_src_svf.res == (target_res_x, target_res_y)}")
                            bounds_match = all(abs(getattr(aligned_src_svf.bounds, attr) - getattr(target_bounds, attr)) < 1e-6 for attr in ['left', 'bottom', 'right', 'top'])
                            print(f"  Aligned SVF Bounds Match: {bounds_match}")
        
                    except subprocess.CalledProcessError as e:
                        print(f"ERROR: GDAL Warp failed for SVF file {os.path.basename(svf_raster_path)}: {e}")
                        print("  STDOUT:", e.stdout)
                        print("  STDERR:", e.stderr)
                    except Exception as e:
                        print(f"An unexpected error occurred during processing SVF file {os.path.basename(svf_raster_path)}: {e}")




print("\n--- Alignment Complete ---")
print(f"Aligned rasters are saved in: {output_dir}")
print(f"Aligned SVF rasters are saved in: {svf_aligned_output_base_dir}")